In [3]:
using LinearAlgebra
using Dates

using Plots
using BenchmarkTools

BLAS.set_num_threads(1)

include("src/io.jl")
include("src/distances.jl")
include("src/optimizer.jl")
include("src/network.jl")
include("src/base.jl")
include("src/pretraining.jl")

# Initialize the parameters
globalParms, MCParms, NNParms, preTrainParms, systemParmsList = parametersInit()

# Initialize the input data
inputs = inputInit(globalParms, NNParms, preTrainParms, systemParmsList)
if globalParms.mode == "training"
    model, opt, refRDFs = inputs
else
    model = inputs
end

Running ML-IMC in the training mode.
Building a model...
Chain(Dense(20 => 20, relu; bias=false), Dense(20 => 20, relu; bias=false), Dense(20 => 1; bias=false))
   Number of layers: 4 
   Number of neurons in each layer: [20, 20, 20, 1]


(Chain(Dense(20 => 20, relu; bias=false), Dense(20 => 20, relu; bias=false), Dense(20 => 1; bias=false)), AMSGrad(0.001, (0.9, 0.999), 1.0e-8, IdDict{Any, Any}()), Any[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  1.004, 1.005, 1.006, 1.005, 1.006, 1.006, 1.007, 1.007, 1.007, 1.007], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  1.003, 1.003, 1.003, 1.004, 1.004, 1.004, 1.005, 1.005, 1.004, 1.005], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  1.004, 1.004, 1.004, 1.005, 1.005, 1.005, 1.005, 1.005, 1.005, 1.005], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  1.004, 1.004, 1.005, 1.004, 1.006, 1.003, 1.004, 1.004, 1.005, 1.004]])

In [4]:
systemParms = systemParmsList[1]

systemParameters("100CH3OH-CG", "methanol-data/100CH3OH/100CH3OH-CG-200.xtc", "methanol-data/100CH3OH/100CH3OH-CG.pdb", 512, "MET", [32.433, 32.433, 32.433], 34116.256126737, "methanol-data/100CH3OH/100CH3OH-CG.rdf", 300, 0.05, 0.0, 298.15, 0.40339559569659034, 1.5, 0.5)

In [5]:
traj = readXTC(systemParms)
frame = read_step(traj, 1)
coordinates = positions(frame)

3×512 Chemfiles.ChemfilesArray:
 18.08  15.83  13.55  14.73  12.49  …  17.62   4.33  24.61   8.32   0.52
  4.76  32.47  12.07  18.75  32.29     14.16  -0.4   11.53  26.35  23.4
 27.28  31.76  13.96   4.45   3.85      3.67  24.95  29.58  21.27  29.59

In [6]:
distanceMatrix = buildDistanceMatrix(frame)

512×512 Matrix{Float64}:
  0.0       7.09189  15.855    17.4607   …   9.68317  15.981    20.8271
  7.09189   0.0      19.4916   14.7913      14.8578   14.2792   17.9268
 15.855    19.4916    0.0      11.6814      19.1468   16.8733   23.2905
 17.4607   14.7913   11.6814    0.0         14.4029   18.7578   16.7659
 12.0278    5.86375  16.1165   13.7371      18.4128   16.9396   16.4657
  6.82342   7.67516  16.2445   13.3721   …  12.6422   17.2571   20.5049
 13.0989   18.7139   17.984    24.9762      11.0264   17.6202   18.7331
 11.3263   10.5723   11.0429   10.3943      14.228    21.6119   20.0425
 19.6562   18.8416   18.4182   13.9998      13.8353   11.5905    6.22577
 15.2346   17.5955   23.1113   24.493       16.9502   11.3124   10.3767
  9.03241  12.3969   11.9791   16.8481   …  15.5818   15.1848   18.55
 15.0417   17.157    15.4381   15.8653      13.3199   15.7629   12.5644
 15.2687   12.0559    9.79153  15.5865      16.0364   18.6892   24.1867
  ⋮                                     

In [8]:
sigma = NNParms.sigma
η = 1 / (2 * sigma * sigma)

0.125